In [1]:
#### Please ignore the man behind the curtain...
#### This is just a hack to add new methods to Python's built-in types.

import ctypes

Py_ssize_t = hasattr(ctypes.pythonapi, "Py_InitModule4_64") and ctypes.c_int64 or ctypes.c_int

class PyObject(ctypes.Structure): pass
PyObject._fields_ = [("ob_refcnt", Py_ssize_t), ("ob_type", ctypes.POINTER(PyObject))]

class SlotsPointer(PyObject):
    _fields_ = [("dict", ctypes.POINTER(PyObject))]

def proxy_builtin(cls):
    name = cls.__name__
    slots = getattr(cls, "__dict__", name)

    pointer = SlotsPointer.from_address(id(slots))
    namespace = {}

    ctypes.pythonapi.PyDict_SetItem(
        ctypes.py_object(namespace),
        ctypes.py_object(name),
        pointer.dict
    )

    return namespace[name]

In [2]:
#### Attach functional methods to the Python "tuple" type (immutable collections).

def mapper(collection):
    """
    Apply a given function to each element of this tuple.
    
    Examples: (1, 2, 3, 4, 5).map(f) == (f(1), f(2), f(3), f(4), f(5))
              (1, 2, 3, 4, 5).map(lambda x: x + 100) == (101, 102, 103, 104, 105)
    """
    return lambda f: tuple(f(x) for x in collection)

def flattener(collection):
    """
    Turn a tuple-of-tuples into a tuple of all elements. Only reduces one level of structure.
    
    Examples: ((1, 2), (3, 4, 5)).flatten == (1, 2, 3, 4, 5)
              ((1, 2), (3, (4, 5))).flatten == (1, 2, 3, (4, 5))
    """
    return sum(collection, ())

def flatmapper(collection):
    """
    Same as tuple.map(f).flatten, but these two operations are frequently done together.
    
    In general: tuple.flatmap(f) == tuple.map(f).flatten
    
    Example: (1, 2, 3, 4, 5).flatmap(lambda x: (x, x + 100)) == (1, 101, 2, 102, 3, 103, 4, 104, 5, 105)
    
    Flatmap is a very general operation. You can use it to expand a table, as above, or to map and filter
    at the same time. (In the theory of monads, "flatmap" is the fundamental "bind" operation.)
    
    Example: (1, 2, 3, 4, 5).flatmap(lambda x: (100 + x,) if x > 2 else ()) == (103, 104, 105)
    
    You might encounter this when you want to compute something for all particles in each event, but also
    handle the case when there are no particles after cuts. In that case, "flatmap" instead of "map" and
    return a singleton tuple (result,) when you have a result and an empty tuple () when you don't.
    """
    return lambda f: sum((f(x) for x in collection), ())

def filterer(collection):
    """
    Apply a given function to each element of the tuple and return only those that returned True.
    
    Example: (1, 2, 3, 4, 5).filter(lambda x: x > 2) == (3, 4, 5)
    """
    return lambda f: tuple(x for x in collection if f(x))

def reducer(collection):
    """
    Apply a given function to each element and a running tally to produce a single result.
    
    Examples: (1, 2, 3, 4, 5).reduce(f) == f(f(f(f(1, 2), 3), 4), 5)
              (1, 2, 3, 4, 5).reduce(lambda x, y: x + y) == 15
    """
    return lambda f: reduce(f, collection)

def aggregator(collection):
    """
    Same as reduce, except start the aggregation on a given zero element.
    
    Examples: (1, 2, 3, 4, 5).aggregate(f, 0) == f(f(f(f(f(0, 1), 2), 3), 4), 5)
              (1, 2, 3, 4, 5).aggregate(lambda x, y: x + y, 0) == 15
              ("a", "b", "c").aggregate(lambda x, y: x + y, "") == "abc"
    """
    return lambda f, zero: reduce(f, collection, zero)

def reducerright(collection):
    """
    Same as reduce, except start the nesting on the right and work left.
    
    Example: (1, 2, 3, 4, 5).reduceright(f) == f(1, f(2, f(3, f(4, 5))))
    """
    return lambda f: reduce(lambda a, b: f(b, a), reversed(collection))

def aggregatorright(collection):
    """
    Same as aggregate, except start the nesting on the right and work left.
    
    Example: (1, 2, 3, 4, 5).aggregateright(f, 0) == f(1, f(2, f(3, f(4, f(5, 0)))))
    """
    return lambda f, zero: reduce(lambda a, b: f(b, a), reversed(collection), zero)

def pairser(collection):
    """
    Apply a given function to pairs of elements without repetition (in either order) or duplicates.
    
    If you think of the input collection as a vector X, this acts on the upper trianglular part of the
    outer product of X with X (not including diagonal).
    
    Alternatively, it's what you would get from these nested loops:
    
        for i in range(len(collection)):
            for j in range(i + 1, len(collection)):   # j starts at i + 1
                f(collection[i], collection[j])
    
    Example: (1, 2, 3, 4, 5).pairs(lambda x, y: (x, y)) == ((1, 2), (1, 3), (1, 4), (1, 5),
                                                                    (2, 3), (2, 4), (2, 5),
                                                                            (3, 4), (3, 5),
                                                                                    (4, 5))
    
    Use this when you want to loop over pairs of distinct pairs of elements from a single collection.
    
    Contrast with "table", which is like a nested loop over several collections, for all elements.
    """
    return lambda f: tuple(f(x, y) for i, x in enumerate(collection) for y in collection[i + 1:])

def tabler(collections):
    """
    """
    def buildargs(first, *rest):
        if len(rest) == 0:
            return tuple((x,) for x in first)
        else:
            return tuple((x,) + y for x in first for y in buildargs(*rest))

    if len(collections) < 2:
        raise TypeError("table requires at least two arguments")
    else:
        first = collections[0]
        rest = collections[1:]
        return lambda f: tuple(f(*args) for args in buildargs(first, *rest))

def zipper(collections):
    if len(collections) < 2:
        raise TypeError("zip requires at least two arguments")
    else:
        return lambda f: tuple(f(*args) for args in zip(*collections))

# attach the methods                                                force Python to notice
proxy_builtin(tuple)["map"] = property(mapper);                     hasattr((), "map")
proxy_builtin(tuple)["flatten"] = property(flattener);              hasattr((), "flatten")
proxy_builtin(tuple)["flatmap"] = property(flatmapper);             hasattr((), "flatmap")
proxy_builtin(tuple)["filter"] = property(filterer);                hasattr((), "filter")
proxy_builtin(tuple)["reduce"] = property(reducer);                 hasattr((), "reduce")
proxy_builtin(tuple)["aggregate"] = property(aggregator);           hasattr((), "aggregate")
proxy_builtin(tuple)["reduceright"] = property(reducerright);       hasattr((), "reduceright")
proxy_builtin(tuple)["aggregateright"] = property(aggregatorright); hasattr((), "aggregateright")
proxy_builtin(tuple)["pairs"] = property(pairser);                  hasattr((), "pairs")
proxy_builtin(tuple)["table"] = property(tabler);                   hasattr((), "table")
proxy_builtin(tuple)["zip"] = property(zipper);                     hasattr((), "zip")



False

In [3]:
assert (1, 2, 3, 4, 5).map(lambda x: 100 + x) == (101, 102, 103, 104, 105)

assert ((1, 2), (3, 4, 5)).flatten == (1, 2, 3, 4, 5)
assert ((1, 2), (3, (4, 5))).flatten == (1, 2, 3, (4, 5))

assert (1, 2, 3, 4, 5).map(lambda x: (x, x)) == ((1, 1), (2, 2), (3, 3), (4, 4), (5, 5))
assert (1, 2, 3, 4, 5).flatmap(lambda x: (x, x + 100)) == (1, 101, 2, 102, 3, 103, 4, 104, 5, 105)
assert (1, 2, 3, 4, 5).flatmap(lambda x: (100 + x,) if x > 2 else ()) == (103, 104, 105)

assert (1, 2, 3, 4, 5).filter(lambda x: x > 2) == (3, 4, 5)

assert (1, 2, 3, 4, 5).reduce(lambda x, y: x + y) == 15
assert (1, 2, 3, 4, 5).reduce(lambda x, y: (x, y)) == ((((1, 2), 3), 4), 5)

assert ("a", "b", "c").aggregate(lambda x, y: x + y, "") == "abc"
assert (1, 2, 3, 4, 5).aggregate(lambda x, y: (x, y), ()) == ((((((), 1), 2), 3), 4), 5)

assert (1, 2, 3, 4, 5).reduceright(lambda x, y: (x, y)) == (1, (2, (3, (4, 5))))

assert (1, 2, 3, 4, 5).aggregateright(lambda x, y: (x, y), ()) == (1, (2, (3, (4, (5, ())))))

assert (1, 2, 3, 4, 5).pairs(lambda x, y: (x, y)) == ((1, 2), (1, 3), (1, 4), (1, 5), (2, 3),
                                                      (2, 4), (2, 5), (3, 4), (3, 5), (4, 5))

assert ((1, 2, 3, 4, 5), ("a", "b")).table(lambda x, y: (x, y)) == (
    (1, "a"), (1, "b"), (2, "a"), (2, "b"), (3, "a"), (3, "b"), (4, "a"), (4, "b"),  (5, "a"), (5, "b"))

assert ((1, 2, 3), ("a", "b", "c"), (101, 102, 103)).zip(lambda x, y, z: (x, y, z)) == (
    (1, "a", 101), (2, "b", 102), (3, "c", 103))